### ÉTAPE 1 - EXPLORATORY DATA ANALYSIS (EDA):

In [207]:
#Importation des bibliothèques de base
#%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
#import matplotlib.pyplot as plt

In [185]:
df1 = pd.read_csv("transaction.csv", delimiter=";")
df1

,02TRANSCATION_ID,ACCOUNT_ID,TRANSCATION_DATE,TRANSCATION_AMOUNT,YEAR,TRANSCATION_AMOUNT_YEAR_SUM
0,T00001,A00866,15/12/2013,37615,2013,15557650
1,T00002,A00106,21/08/2018,9212,2018,77495196
2,T00003,A00233,25/12/2020,62526,2020,76208785
3,T00004,A00848,01/10/2021,90070,2021,71973924
4,T00005,A00404,06/09/2018,80153,2018,77495196
...,...,...,...,...,...,...
14995,T04996,A00882,09/04/2022,54893,2022,73969670
14996,T04997,A00440,25/09/2017,30120,2017,75243074
14997,T04998,A00332,07/04/2023,95138,2023,51646874
14998,T04999,A00497,15/03/2022,31009,2022,73969670


In [186]:
import pandas as pd

# Chargez les données dans un DataFrame
df = pd.read_csv("transaction.csv", delimiter=";")
df=df.drop(columns=['YEAR','TRANSCATION_AMOUNT_YEAR_SUM'])
# Convert TRANSCATION_DATE column to datetime type
df['TRANSCATION_DATE'] = pd.to_datetime(df['TRANSCATION_DATE'])
# Rename the miswritten columns
df = df.rename(columns={'02TRANSCATION_ID': 'TRANSACTION_ID',
                        'TRANSCATION_DATE': 'TRANSACTION_DATE',
                        'TRANSCATION_AMOUNT': 'TRANSACTION_AMOUNT'})
df


,TRANSACTION_ID,ACCOUNT_ID,TRANSACTION_DATE,TRANSACTION_AMOUNT
0,T00001,A00866,2013-12-15,37615
1,T00002,A00106,2018-08-21,9212
2,T00003,A00233,2020-12-25,62526
3,T00004,A00848,2021-10-01,90070
4,T00005,A00404,2018-09-06,80153
...,...,...,...,...
14995,T04996,A00882,2022-04-09,54893
14996,T04997,A00440,2017-09-25,30120
14997,T04998,A00332,2023-04-07,95138
14998,T04999,A00497,2022-03-15,31009


In [187]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df,title= 'DATA TRANSACTIONS')
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [188]:
profile.to_file('data_report.html')

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [189]:
## Extract year and month from the TRANSACTION_DATE column
df['TRANSACTION_YEAR_MONTH'] = df['TRANSACTION_DATE'].dt.to_period('M')

# Group by year and month and sum transaction amount
aggregated_data = df.groupby('TRANSACTION_YEAR_MONTH')['TRANSACTION_AMOUNT'].sum().reset_index()
aggregated_data

,TRANSACTION_YEAR_MONTH,TRANSACTION_AMOUNT
0,2013-10,2808140
1,2013-11,6362394
2,2013-12,6387116
3,2014-01,5624575
4,2014-02,4618111
...,...,...
114,2023-04,6771888
115,2023-05,6415633
116,2023-06,6402621
117,2023-07,6830486


In [190]:



# Convert TRANSACTION_YEAR_MONTH to string
aggregated_data['TRANSACTION_YEAR_MONTH'] = aggregated_data['TRANSACTION_YEAR_MONTH'].astype(str)

# Create a line plot using Plotly graph_objects
fig = go.Figure(data=go.Scatter(x=aggregated_data['TRANSACTION_YEAR_MONTH'], 
                                y=aggregated_data['TRANSACTION_AMOUNT'], 
                                mode='lines+markers'))

# Update layout
fig.update_layout(title='Monthly Transaction Amount',
                  xaxis_title='Month',
                  yaxis_title='Total Transaction Amount')

fig.show()

In [191]:
df=aggregated_data.copy()
df = df[1:-1]
df

,TRANSACTION_YEAR_MONTH,TRANSACTION_AMOUNT
1,2013-11,6362394
2,2013-12,6387116
3,2014-01,5624575
4,2014-02,4618111
5,2014-03,5686054
...,...,...
113,2023-03,6443150
114,2023-04,6771888
115,2023-05,6415633
116,2023-06,6402621


### statistic analysis of the timeserie :


### ADF

In [192]:
from statsmodels.tsa.stattools import adfuller
# Perform ADF test for each range time series
adf_results = {}


# Perform ADF test
result = adfuller(df['TRANSACTION_AMOUNT'])

# Determine stationarity based on p-value
is_stationary = result[1] < 0.05

# Extract ADF test results
adf_results = {
    'ADF Statistic': result[0],
    'p-value': result[1],
    'Critical Values': result[4],
    'Stationary': is_stationary
}

# Print ADF test results

print(f"ADF Test Results :")
for key, value in adf_results.items():
    print(f"{key}: {value}")
print()


ADF Test Results :
ADF Statistic: -10.774490917148189
p-value: 2.3385815847363295e-19
Critical Values: {'1%': -3.4880216384691867, '5%': -2.8867966864160075, '10%': -2.5802408234244947}
Stationary: True



### KPSS

In [193]:
from statsmodels.tsa.stattools import kpss

# Define a function to perform the KPSS test and print the results
def kpss_test_stationarity(data):
    # Perform the KPSS test
    result = kpss(data, regression='ct')
    
    # Print test statistic and p-value
    print('KPSS Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    
    # Print test result
    if result[1] < 0.05:
        print("The series is non-stationary (reject the null hypothesis)")
    else:
        print("The series is stationary (fail to reject the null hypothesis)")

# Perform KPSS test for each range time series
kpss_results = {}


print(f"KPSS Test Results :")
kpss_test_stationarity(df['TRANSACTION_AMOUNT'])
print()


KPSS Test Results :
KPSS Statistic: 0.035886
p-value: 0.100000
The series is stationary (fail to reject the null hypothesis)



### autocorrelation (acf, pacf)

In [194]:
import numpy as np
import plotly.graph_objs as go

from statsmodels.tsa.stattools import acf, pacf

def create_acf_plot(series):
    corr_array = acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines', line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,25])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Autocorrelation (ACF)'
    fig.update_layout(title=title)
    fig.show()

def create_pacf_plot(series):
    corr_array = pacf(series.dropna(), alpha=0.05) 
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]

    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines', line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,25])
    fig.update_yaxes(zerolinecolor='#000000')
    
    title='Partial Autocorrelation (PACF)' 
    fig.update_layout(title=title)
    fig.show()  


In [195]:
print(f"Autocorrelation Function (ACF) :")
create_acf_plot(df['TRANSACTION_AMOUNT'])

print(f"Partial Autocorrelation Function (PACF):")
create_pacf_plot(df['TRANSACTION_AMOUNT'])


Autocorrelation Function (ACF) :


Partial Autocorrelation Function (PACF):


### seasonal decompose

In [205]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose



# Convert TRANSACTION_YEAR_MONTH to string
df['TRANSACTION_YEAR_MONTH'] = df['TRANSACTION_YEAR_MONTH'].astype(str)

# Perform seasonal decomposition
result = seasonal_decompose(df['TRANSACTION_AMOUNT'], model='additive', period=12)

# Create traces for each component
trend_trace = go.Scatter(x=df['TRANSACTION_YEAR_MONTH'], y=result.trend, mode='lines', name='Trend')
seasonal_trace = go.Scatter(x=df['TRANSACTION_YEAR_MONTH'], y=result.seasonal, mode='lines', name='Seasonal')
residual_trace = go.Scatter(x=df['TRANSACTION_YEAR_MONTH'], y=result.resid, mode='lines', name='Residual')

# Create subplot grid
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                    subplot_titles=('Trend', 'Seasonal', 'Residual'))

# Add traces to subplots
fig.add_trace(trend_trace, row=1, col=1)
fig.add_trace(seasonal_trace, row=2, col=1)
fig.add_trace(residual_trace, row=3, col=1)

# Update layout
fig.update_layout(title='Seasonal Decomposition',
                  xaxis_title='Month',
                  yaxis_title='Amount',
                  height=800, width=800,
                  template='plotly_white')

fig.show()

In [197]:
df

,TRANSACTION_YEAR_MONTH,TRANSACTION_AMOUNT
1,2013-11,6362394
2,2013-12,6387116
3,2014-01,5624575
4,2014-02,4618111
5,2014-03,5686054
...,...,...
113,2023-03,6443150
114,2023-04,6771888
115,2023-05,6415633
116,2023-06,6402621


## MODELING

In [198]:
import pandas as pd
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler


# Convert TRANSACTION_YEAR_MONTH to datetime
df['TRANSACTION_YEAR_MONTH'] = pd.to_datetime(df['TRANSACTION_YEAR_MONTH'])
# Scale the data
scaler = MinMaxScaler()
df['TRANSACTION_AMOUNT_SCALED'] = scaler.fit_transform(df[['TRANSACTION_AMOUNT']])



# Split the data into training and testing sets
train_data = df[:-12]  # All data except the last 12 months
test_data = df[-12:]   # Last 12 months

# Fit auto ARIMA model
model = auto_arima(train_data['TRANSACTION_AMOUNT_SCALED'], d=0,
                   seasonal=True,trace=True,m=7,
                      error_action='ignore',  
                      suppress_warnings=True, 
                     stepwise=True,with_intercept=True,trend='t')

# Predict 12 steps ahead
predictions = model.predict(n_periods=12)
# Visualize historical data, predictions, and true values
fig = go.Figure()

# Historical data
fig.add_trace(go.Scatter(x=train_data['TRANSACTION_YEAR_MONTH'], y=train_data['TRANSACTION_AMOUNT_SCALED'],
                         mode='lines', name='Historical Data'))
# True values
fig.add_trace(go.Scatter(x=test_data['TRANSACTION_YEAR_MONTH'], y=test_data['TRANSACTION_AMOUNT_SCALED'],
                         mode='lines', name='True Values'))
# Predictions
future_dates = pd.date_range(start=test_data['TRANSACTION_YEAR_MONTH'].iloc[0], periods=12, freq='MS')
fig.add_trace(go.Scatter(x=future_dates, y=predictions,
                         mode='lines', name='Predictions'))

# Update layout
fig.update_layout(title='Historical Data, Predictions, and True Values',
                  xaxis_title='Date',
                  yaxis_title='Transaction Amount')

fig.show()



# Calculate MSE, RMSE, and MFE
mse = mean_squared_error(test_data['TRANSACTION_AMOUNT_SCALED'], predictions)
rmse = np.sqrt(mse)
mfe = np.mean(predictions - test_data['TRANSACTION_AMOUNT_SCALED'])

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Forecast Error (MFE):", mfe)


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[7] intercept   : AIC=15.183, Time=0.38 sec
 ARIMA(0,0,0)(0,0,0)[7] intercept   : AIC=69.016, Time=0.02 sec
 ARIMA(1,0,0)(1,0,0)[7] intercept   : AIC=20.373, Time=0.24 sec
 ARIMA(0,0,1)(0,0,1)[7] intercept   : AIC=39.488, Time=0.11 sec
 ARIMA(0,0,0)(0,0,0)[7]             : AIC=69.016, Time=0.02 sec
 ARIMA(2,0,2)(0,0,1)[7] intercept   : AIC=-5.679, Time=0.34 sec
 ARIMA(2,0,2)(0,0,0)[7] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(2,0,2)(0,0,2)[7] intercept   : AIC=10.937, Time=0.43 sec
 ARIMA(2,0,2)(1,0,0)[7] intercept   : AIC=13.792, Time=0.24 sec
 ARIMA(2,0,2)(1,0,2)[7] intercept   : AIC=17.828, Time=0.52 sec
 ARIMA(1,0,2)(0,0,1)[7] intercept   : AIC=-2.602, Time=0.15 sec
 ARIMA(2,0,1)(0,0,1)[7] intercept   : AIC=-1.511, Time=0.20 sec
 ARIMA(3,0,2)(0,0,1)[7] intercept   : AIC=-1.994, Time=0.30 sec
 ARIMA(2,0,3)(0,0,1)[7] intercept   : AIC=1.778, Time=0.28 sec
 ARIMA(1,0,1)(0,0,1)[7] intercept   : AIC=-2.155, Time=0.17 sec
 

Mean Squared Error (MSE): 0.03590596346059782
Root Mean Squared Error (RMSE): 0.1894886895321138
Mean Forecast Error (MFE): 0.0154073317404451


In [199]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
train_data = df[:-12]  # All data except the last 12 months
test_data = df[-12:]   # Last 12 months

# Initialize an empty list to store predictions
predictions = []

# Fit auto ARIMA model
train=train_data['TRANSACTION_AMOUNT_SCALED']
# Predict one step ahead iteratively for 12 steps
for i in range(12):
    model = SARIMAX(train, order=(2,0,0), seasonal_order=(1,0,0,12))
    fit_model = model.fit(disp=False)
    # Predict one step ahead
    forecast = fit_model.forecast(steps=1)

    # Append the prediction to the list
    predictions.append(list(forecast)[0])
    next_true_value = test_data.iloc[i]['TRANSACTION_AMOUNT_SCALED']
    next_true_value = next_true_value.astype(float)
    # Append the true value for the next step
    train=pd.concat([train_data['TRANSACTION_AMOUNT_SCALED'], pd.Series([next_true_value], index=[test_data.index[i]])])

# Visualize historical data, predictions, and true values
fig = go.Figure()

# Historical data
fig.add_trace(go.Scatter(x=train_data['TRANSACTION_YEAR_MONTH'], y=train_data['TRANSACTION_AMOUNT_SCALED'],
                         mode='lines', name='Historical Data'))
# True values
fig.add_trace(go.Scatter(x=test_data['TRANSACTION_YEAR_MONTH'], y=test_data['TRANSACTION_AMOUNT_SCALED'],
                         mode='lines', name='True Values'))
# Predictions
future_dates = pd.date_range(start=test_data['TRANSACTION_YEAR_MONTH'].iloc[0], periods=12, freq='MS')
fig.add_trace(go.Scatter(x=future_dates, y=predictions,
                         mode='lines', name='Predictions'))

# Update layout
fig.update_layout(title='Historical Data, Predictions, and True Values',
                  xaxis_title='Date',
                  yaxis_title='Transaction Amount')

fig.show()
# Calculate MSE, RMSE, and MFE
mse = mean_squared_error(test_data['TRANSACTION_AMOUNT_SCALED'], predictions)
rmse = np.sqrt(mse)
mfe = np.mean(predictions - test_data['TRANSACTION_AMOUNT_SCALED'])

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Forecast Error (MFE):", mfe)


Mean Squared Error (MSE): 0.05731051856845502
Root Mean Squared Error (RMSE): 0.23939615403856224
Mean Forecast Error (MFE): 0.007435427400550143


In [206]:
df

,TRANSACTION_YEAR_MONTH,TRANSACTION_AMOUNT,TRANSACTION_AMOUNT_SCALED,Month,Year,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5,Lag_6,Lag_7
8,2014-06-01,6896485,0.663662,6,2014,0.829779,0.619004,0.311079,0.000000,0.293171,0.515289,0.508088
9,2014-07-01,6586198,0.573280,7,2014,0.663662,0.829779,0.619004,0.311079,0.000000,0.293171,0.515289
10,2014-08-01,6207581,0.462993,8,2014,0.573280,0.663662,0.829779,0.619004,0.311079,0.000000,0.293171
11,2014-09-01,5677598,0.308616,9,2014,0.462993,0.573280,0.663662,0.829779,0.619004,0.311079,0.000000
12,2014-10-01,6200222,0.460849,10,2014,0.308616,0.462993,0.573280,0.663662,0.829779,0.619004,0.311079
...,...,...,...,...,...,...,...,...,...,...,...,...
113,2023-03-01,6443150,0.531611,3,2023,0.255524,1.000000,0.384334,0.307269,0.276720,0.310414,0.624246
114,2023-04-01,6771888,0.627369,4,2023,0.531611,0.255524,1.000000,0.384334,0.307269,0.276720,0.310414
115,2023-05-01,6415633,0.523596,5,2023,0.627369,0.531611,0.255524,1.000000,0.384334,0.307269,0.276720
116,2023-06-01,6402621,0.519806,6,2023,0.523596,0.627369,0.531611,0.255524,1.000000,0.384334,0.307269


In [201]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Create additional features: month of year, year, and 7 lagged values
df['Month'] = df['TRANSACTION_YEAR_MONTH'].dt.month
df['Year'] = df['TRANSACTION_YEAR_MONTH'].dt.year
for i in range(1, 8):
    df[f'Lag_{i}'] = df['TRANSACTION_AMOUNT_SCALED'].shift(i)

# Drop rows with NaN values due to lagged features
df.dropna(inplace=True)

# Split the data into features and target variable
X = df.drop(['TRANSACTION_AMOUNT','TRANSACTION_YEAR_MONTH', 'TRANSACTION_AMOUNT_SCALED'], axis=1)
y = df['TRANSACTION_AMOUNT_SCALED']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=12, shuffle=False)

# Define XGBoost model
model = xgb.XGBRegressor()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate MSE, RMSE, and MFE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mfe = np.mean(y_pred - y_test)

print("MSE:", mse)
print("RMSE:", rmse)
print("MFE:", mfe)

# Visualize historical, true, and predicted values
fig = go.Figure()

# Historical values
fig.add_trace(go.Scatter(x=df['TRANSACTION_YEAR_MONTH'], y=df['TRANSACTION_AMOUNT_SCALED'],
                         mode='lines', name='Historical'))

# True values
fig.add_trace(go.Scatter(x=df['TRANSACTION_YEAR_MONTH'][-len(y_test):], y=y_test,
                         mode='lines', name='True Values'))

# Predictions
fig.add_trace(go.Scatter(x=df['TRANSACTION_YEAR_MONTH'][-len(y_test):], y=y_pred,
                         mode='lines', name='Predictions'))

# Update layout
fig.update_layout(title='Historical vs True vs Predicted Values',
                  xaxis_title='Date',
                  yaxis_title='Transaction Amount (Scaled)')
fig.show()

MSE: 0.04262152279171727
RMSE: 0.20644980695490434
MFE: 0.00453197138827723


In [202]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import plotly.graph_objects as go



# Drop rows with NaN values due to lagged features
df.dropna(inplace=True)

# Split the data into features and target variable
X = df.drop(['TRANSACTION_AMOUNT','TRANSACTION_YEAR_MONTH', 'TRANSACTION_AMOUNT_SCALED'], axis=1)
y = df['TRANSACTION_AMOUNT_SCALED']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=12, shuffle=False)

# Define LightGBM model
model = lgb.LGBMRegressor()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate MSE, RMSE, and MFE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mfe = np.mean(y_pred - y_test)

print("MSE:", mse)
print("RMSE:", rmse)
print("MFE:", mfe)

# Visualize historical, true, and predicted values
fig = go.Figure()

# Historical values
fig.add_trace(go.Scatter(x=df['TRANSACTION_YEAR_MONTH'], y=df['TRANSACTION_AMOUNT_SCALED'],
                         mode='lines', name='Historical'))

# True values
fig.add_trace(go.Scatter(x=df['TRANSACTION_YEAR_MONTH'][-len(y_test):], y=y_test,
                         mode='lines', name='True Values'))

# Predictions
fig.add_trace(go.Scatter(x=df['TRANSACTION_YEAR_MONTH'][-len(y_test):], y=y_pred,
                         mode='lines', name='Predictions'))

# Update layout
fig.update_layout(title='Historical vs True vs Predicted Values (LightGBM)',
                  xaxis_title='Date',
                  yaxis_title='Transaction Amount (Scaled)')
fig.show()


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000039 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261
[LightGBM] [Info] Number of data points in the train set: 98, number of used features: 9
[LightGBM] [Info] Start training from score 0.502100
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [203]:
# Split the data into features and target variable
X = df.drop(['TRANSACTION_AMOUNT','TRANSACTION_YEAR_MONTH', 'TRANSACTION_AMOUNT_SCALED'], axis=1)
y = df['TRANSACTION_AMOUNT_SCALED']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=12, shuffle=False)

# Define XGBoost model
xgb_model = xgb.XGBRegressor()

# Fit XGBoost model
xgb_model.fit(X_train, y_train)

# Make XGBoost predictions
xgb_pred = xgb_model.predict(X_test)

# Define LightGBM model
lgb_model = lgb.LGBMRegressor()

# Fit LightGBM model
lgb_model.fit(X_train, y_train)

# Make LightGBM predictions
lgb_pred = lgb_model.predict(X_test)

# Ensemble predictions by averaging
ensemble_pred = (xgb_pred + lgb_pred) / 2

# Calculate ensemble MSE, RMSE, and MFE
mse = mean_squared_error(y_test, ensemble_pred)
rmse = np.sqrt(mse)
mfe = np.mean(ensemble_pred - y_test)

print("Ensemble MSE:", mse)
print("Ensemble RMSE:", rmse)
print("Ensemble MFE:", mfe)

# Visualize historical, true, and ensemble predicted values
fig = go.Figure()

# Historical values
fig.add_trace(go.Scatter(x=df['TRANSACTION_YEAR_MONTH'], y=df['TRANSACTION_AMOUNT_SCALED'],
                         mode='lines', name='Historical'))

# True values
fig.add_trace(go.Scatter(x=df['TRANSACTION_YEAR_MONTH'][-len(y_test):], y=y_test,
                         mode='lines', name='True Values'))

# Ensemble Predictions
fig.add_trace(go.Scatter(x=df['TRANSACTION_YEAR_MONTH'][-len(y_test):], y=ensemble_pred,
                         mode='lines', name='Ensemble Predictions'))

# Update layout
fig.update_layout(title='Historical vs True vs Ensemble Predicted Values',
                  xaxis_title='Date',
                  yaxis_title='Transaction Amount (Scaled)')
fig.show()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000042 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261
[LightGBM] [Info] Number of data points in the train set: 98, number of used features: 9
[LightGBM] [Info] Start training from score 0.502100
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best